# ElementAIR Kits analysis

In [ ]:
import scdata as sc

In [ ]:
from scdata._config import config

config._outlevel = 'DEBUG'

In [ ]:
from scdata.io.device_api import ScApiDevice

In [ ]:
import seaborn as sns

### Find sensors

In [ ]:
sensors = ScApiDevice.global_search(value='ElementAIR_6', full=True)
sensors

In [ ]:
devices = sensors.index.tolist()
devices

In [ ]:
test = sc.Test('UCLL_KITS_ANALYSIS')

### Create test

In [ ]:
for device in devices:
    test.add_device(sc.Device(blueprint = 'sck_21', descriptor = {'source': 'api',
                                                  'id': device,
                                                  'frequency': '1Min',
                                                  'location': 'Europe/Madrid'}))

In [ ]:
test.create()

### Load Data

In [ ]:
test.load()

### Basic exploration

In [ ]:
traces = {1: {'devices': 'all', 'channel': 'TEMP', 'subplot': 1}
         }

formatting = {'width': 1000, 'title': 'Temperature', 'ylabel': {1: 'Temperature (degC)'}}
options = {'clean_na': 'None', 'frequency': '5Min'}

iframe = test.ts_uplot(traces = traces, formatting = formatting, options = options)

test.add_content(title = 'Temperature graph', iframe = iframe)
iframe

In [ ]:
traces = {1: {'devices': 'all', 'channel': 'NOISE_A', 'subplot': 1}
         }

formatting = {'width': 1000, 'title': 'Noise', 'ylabel': {1: 'Noise (dBA)'}}
options = {'clean_na': 'None', 'frequency': '5Min'}

iframe = test.ts_uplot(traces = traces, formatting = formatting, options = options)
test.add_content(title = 'Noise graph', iframe = iframe, text = 'Please, note here the effect of the noise floor as mentioned')
iframe

In [ ]:
traces = {1: {'devices': 'all', 'channel': 'PM_25', 'subplot': 1}
         }

formatting = {'width': 1000, 'title': 'PM 2.5', 'ylabel': {1: 'PM 2.5 (ug/m3)'}}
options = {'clean_na': 'None', 'frequency': '5Min'}

iframe = test.ts_uplot(traces = traces, formatting = formatting, options = options)
test.add_content(title = 'PM 2.5 graph', iframe = iframe)
iframe

In [ ]:
traces = {
            "1": {"devices": "13167",
                  "channel": "NOISE_A",
                  "subplot": 1},
        }

options = {'frequency':'1H', 'clean_na': 'drop', 'frequency_hours': 6, 'session': '1D'}
formatting = {'width': 28}

figure = test.heatmap_plot(traces = traces, options = options, formatting = formatting);

test.add_content(title = 'Noise Visualisations (ElementAIR_6A_1)', figure = figure)

In [ ]:
traces = {
            "1": {"devices": "13172",
                  "channel": "NOISE_A",
                  "subplot": 1},
        }

options = {'frequency':'1H', 'clean_na': 'drop', 'frequency_hours': 6, 'session': '1D'}
formatting = {'width': 28}

figure = test.heatmap_plot(traces = traces, options = options, formatting = formatting);
test.add_content(title = 'Noise Visualisations (ElementAIR_6C_2)', figure = figure)

## Agregated data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
combined_df = pd.concat([test.devices[str(device)].readings for device in devices])

In [ ]:
combined_df['device'] = np.repeat([str(device) for device in devices], [len(test.devices[str(device)].readings) for device in devices])

In [ ]:
combined_df['day_of_week'] = combined_df.index.day_name()

In [ ]:
def is_business(x):
    if x['day_of_week'] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']: return True
    else: return False
    
combined_df['is_business'] = combined_df.apply(lambda x: is_business(x), axis = 1)
combined_df.head(5)

In [ ]:
g = sns.catplot(x="is_business", y="PM_25",
            col = "device", col_wrap=3,
            data=combined_df, height=8, aspect=0.8)

(g.set_axis_labels("", "PM 2.5 (ug/m3)")
  .set_xticklabels(["Weekends", "Mondays to Fridays"])
# .set(ylim=(0, 80))
  .despine(left=True))

test.add_content(title= "PM 2.5 weekday comparison", figure = g)

In [ ]:
g = sns.catplot(x="is_business", y="NOISE_A",
            col = "device", col_wrap=3,
            data=combined_df, height=8, aspect=0.8)

(g.set_axis_labels("", "Noise Level (dBA)")
  .set_xticklabels(["Weekends", "Mondays to Fridays"])
# .set(ylim=(0, 80))
  .despine(left=True))

test.add_content(title= "Noise weekday comparison", figure = g, text = 'Noise levels are clearly much higher in every case')

## Export in html

In [ ]:
test.to_html(title = 'ElementAIR sensors', details=False);

## Simplified data

In [ ]:
for device in devices:
    df = test.devices[str(device)].readings.copy()
    df = df.resample('1H').mean()
    
    df.to_csv(f'~/Desktop/{device}.csv')